In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote 
from datetime import timedelta

# Get joining table

In [2]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

# using sql query to get recent date
sql2 ='''SELECT dividend_info."div_announced_year",dividend_info."div_financial_year",
dividend_info."div_subject",dividend_info."div_ex_date",dividend_info."div_payment_date"
,dividend_info."div_dividend(RM)"
,dividend_info."div_indicator",dividend_info."div_stock_code",dividend_info."div_year",
dividend_info."div_status",daily_stock."stock_date",daily_stock."stock_code",
daily_stock."stock_close"
FROM dividend_info  LEFT JOIN daily_stock  
ON (dividend_info."div_stock_code"=daily_stock."stock_code" AND dividend_info."div_ex_date"=daily_stock."stock_date")
ORDER BY dividend_info."div_ex_date";'''
cursor.execute(sql2)
daily_stock = cursor.fetchall()
#print(recent_date[0])
df = pd.DataFrame(daily_stock, columns =["div_announced_year","div_financial_year","div_subject","div_ex_date",
                                         "div_payment_date","div_dividend(RM)","div_indicator","div_stock_code",
                                         "div_year","div_status","stock_date","stock_code","stock_close"])
conn.commit()
conn.close()

In [3]:
df

,div_announced_year,div_financial_year,div_subject,div_ex_date,div_payment_date,div_dividend(RM),div_indicator,div_stock_code,div_year,div_status,stock_date,stock_code,stock_close
0,2017-06-14,2018-01-31,First Interim Dividend,2017-06-30,2017-07-14,0.030,Currency,6399,2018,Y,2017-06-30,6399,1.781
1,2017-06-20,2018-04-30,Interim Dividend,2017-07-05,2017-07-28,0.015,Currency,7235,2018,Y,2017-07-05,7235,2.014
2,2017-07-04,2018-03-31,Interim Dividend,2017-07-14,2017-07-28,0.010,Currency,5267,2018,Y,2017-07-14,5267,1.208
3,2017-07-13,2018-02-28,First Interim Dividend,2017-07-27,2017-08-23,0.060,Currency,7048,2018,Y,2017-07-27,7048,3.519
4,2017-08-10,2018-03-31,Interim Dividend,2017-08-23,2017-09-08,0.020,Currency,0035,2018,Y,2017-08-23,0035,0.568
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5781,None,None,None,None,None,0.000,None,5256,2021,N,None,None,NaN
5782,None,None,None,None,None,0.000,None,5256,2020,N,None,None,NaN
5783,None,None,None,None,None,0.000,None,5256,2019,N,None,None,NaN
5784,None,None,None,None,None,0.000,None,5256,2018,N,None,None,NaN


In [4]:
# delete unneccessary columns
del df['div_subject']
del df['div_payment_date']
del df['div_indicator']
del df['stock_code']
del df['stock_date']
del df['div_financial_year']

In [5]:
df

,div_announced_year,div_ex_date,div_dividend(RM),div_stock_code,div_year,div_status,stock_close
0,2017-06-14,2017-06-30,0.030,6399,2018,Y,1.781
1,2017-06-20,2017-07-05,0.015,7235,2018,Y,2.014
2,2017-07-04,2017-07-14,0.010,5267,2018,Y,1.208
3,2017-07-13,2017-07-27,0.060,7048,2018,Y,3.519
4,2017-08-10,2017-08-23,0.020,0035,2018,Y,0.568
...,...,...,...,...,...,...,...
5781,None,None,0.000,5256,2021,N,NaN
5782,None,None,0.000,5256,2020,N,NaN
5783,None,None,0.000,5256,2019,N,NaN
5784,None,None,0.000,5256,2018,N,NaN


In [6]:
df.dtypes

div_announced_year     object
div_ex_date            object
div_dividend(RM)      float64
div_stock_code         object
div_year               object
div_status             object
stock_close           float64
dtype: object

In [7]:
df['div_announced_year'] = pd.to_datetime(df['div_announced_year'] )
df['div_announced_year']  = pd.to_datetime(df['div_announced_year'] .dt.strftime('%Y-%m-%d'))

df['div_ex_date'] = pd.to_datetime(df['div_ex_date'] )
df['div_ex_date']  = pd.to_datetime(df['div_ex_date'].dt.strftime('%Y-%m-%d'))



In [8]:
df.dtypes

div_announced_year    datetime64[ns]
div_ex_date           datetime64[ns]
div_dividend(RM)             float64
div_stock_code                object
div_year                      object
div_status                    object
stock_close                  float64
dtype: object

In [9]:
df['dividend_y_%']=(df['div_dividend(RM)']/df['stock_close'])*100

In [10]:
df

,div_announced_year,div_ex_date,div_dividend(RM),div_stock_code,div_year,div_status,stock_close,dividend_y_%
0,2017-06-14,2017-06-30,0.030,6399,2018,Y,1.781,1.684447
1,2017-06-20,2017-07-05,0.015,7235,2018,Y,2.014,0.744786
2,2017-07-04,2017-07-14,0.010,5267,2018,Y,1.208,0.827815
3,2017-07-13,2017-07-27,0.060,7048,2018,Y,3.519,1.705030
4,2017-08-10,2017-08-23,0.020,0035,2018,Y,0.568,3.521127
...,...,...,...,...,...,...,...,...
5781,NaT,NaT,0.000,5256,2021,N,NaN,NaN
5782,NaT,NaT,0.000,5256,2020,N,NaN,NaN
5783,NaT,NaT,0.000,5256,2019,N,NaN,NaN
5784,NaT,NaT,0.000,5256,2018,N,NaN,NaN


In [11]:
df['dividend_y_%']=df['dividend_y_%'].round(3)

In [12]:
quarter_div_info = df

In [13]:
fun={'dividend_y_%':'sum','div_dividend(RM)':'sum'}
annual_div_info=df.groupby(['div_stock_code','div_year','div_status']).agg(fun)
annual_div_info.head(10)

dividend_y_%  div_dividend(RM)
div_stock_code div_year div_status                                
0001           2018     Y                  1.650             0.015
               2019     Y                  1.159             0.015
               2020     Y                  0.971             0.015
               2021     Y                  1.343             0.015
               2022     Y                  0.282             0.005
0002           2018     Y                  3.396             0.050
               2019     Y                  4.306             0.074
               2020     Y                  3.847             0.090
               2021     Y                  3.167             0.090
               2022     Y                  5.418             0.255

In [14]:
annual_div_info= annual_div_info.reset_index()

In [15]:
annual_div_info

,div_stock_code,div_year,div_status,dividend_y_%,div_dividend(RM)
0,0001,2018,Y,1.650,0.0150
1,0001,2019,Y,1.159,0.0150
2,0001,2020,Y,0.971,0.0150
3,0001,2021,Y,1.343,0.0150
4,0001,2022,Y,0.282,0.0050
...,...,...,...,...,...
3950,9962,2020,Y,2.101,0.0050
3951,9962,2021,Y,3.362,0.0100
3952,9962,2022,Y,1.754,0.0050
3953,9997,2021,Y,3.716,0.0225


In [ ]:
quarter_div_info.to_csv("quarter_div_info.csv")
annual_div_info.to_csv("annual_div_info.csv")

#

In [16]:
annual= pd.read_csv('annual_div_info.csv')

In [17]:
annual

,Unnamed: 0,div_stock_code,div_year,div_status,dividend_y_%,div_dividend(RM)
0,0,0001,2018,Y,1.650,0.0150
1,1,0001,2019,Y,1.159,0.0150
2,2,0001,2020,Y,0.971,0.0150
3,3,0001,2021,Y,1.343,0.0150
4,4,0001,2022,Y,0.282,0.0050
...,...,...,...,...,...,...
3950,3950,9962,2020,Y,2.101,0.0050
3951,3951,9962,2021,Y,3.362,0.0100
3952,3952,9962,2022,Y,1.754,0.0050
3953,3953,9997,2021,Y,3.716,0.0225


In [18]:
del annual['Unnamed: 0']

In [19]:
annual

,div_stock_code,div_year,div_status,dividend_y_%,div_dividend(RM)
0,0001,2018,Y,1.650,0.0150
1,0001,2019,Y,1.159,0.0150
2,0001,2020,Y,0.971,0.0150
3,0001,2021,Y,1.343,0.0150
4,0001,2022,Y,0.282,0.0050
...,...,...,...,...,...
3950,9962,2020,Y,2.101,0.0050
3951,9962,2021,Y,3.362,0.0100
3952,9962,2022,Y,1.754,0.0050
3953,9997,2021,Y,3.716,0.0225


In [20]:
dpr=pd.read_csv('div_annual_report(Final).csv')

In [21]:
dpr

,Unnamed: 0,annual_financial_year,annual_revenue('000),annual_net_income('000),annual_eps,annual_dp(%),annual_stock_code
0,0,2022-12-31,2771856,63803,5.66,NaN,5250
1,1,2021-12-31,2809085,43483,3.86,NaN,5250
2,2,2020-12-31,2537563,29767,2.60,NaN,5250
3,3,2019-12-31,2359399,54059,4.74,NaN,5250
4,4,2018-12-31,2216099,51307,4.56,NaN,5250
...,...,...,...,...,...,...,...
5212,5212,2022-11-30,136167,42487,8.33,2.04,5131
5213,5213,2021-11-30,149616,45360,9.86,1.72,5131
5214,5214,2020-11-30,170678,46773,10.17,1.67,5131
5215,5215,2019-11-30,170622,49522,10.77,1.11,5131


In [22]:
del dpr['Unnamed: 0']


In [25]:
dpr['annual_financial_year'] = pd.to_datetime(dpr['annual_financial_year']  )
dpr['annual_financial_year']   = pd.to_datetime(dpr['annual_financial_year'] .dt.strftime('%Y-%m-%d'))

dpr['annual_year'] = dpr['annual_financial_year'].dt.strftime('%Y')

In [28]:
dpr

,annual_financial_year,annual_revenue('000),annual_net_income('000),annual_eps,annual_dp(%),annual_stock_code,annual_year
0,2022-12-31,2771856,63803,5.66,NaN,5250,2022
1,2021-12-31,2809085,43483,3.86,NaN,5250,2021
2,2020-12-31,2537563,29767,2.60,NaN,5250,2020
3,2019-12-31,2359399,54059,4.74,NaN,5250,2019
4,2018-12-31,2216099,51307,4.56,NaN,5250,2018
...,...,...,...,...,...,...,...
5212,2022-11-30,136167,42487,8.33,2.04,5131,2022
5213,2021-11-30,149616,45360,9.86,1.72,5131,2021
5214,2020-11-30,170678,46773,10.17,1.67,5131,2020
5215,2019-11-30,170622,49522,10.77,1.11,5131,2019
